In [36]:
path = "./mrqa-few-shot/bioasq/bioasq-train-seed-42-num-examples-1024.jsonl"

In [37]:
path2 = "./mrqa-few-shot/bioasq/dev.jsonl"

In [38]:
import json 

with open(path2, 'r') as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)

#### First lets create the validation_set.

This only needs to be done once.

In [1]:
from datasets import load_dataset
import pandas as pd
import json

dataset = load_dataset("squad", split="validation")
val_df = pd.DataFrame(dataset)

Found cached dataset squad (/Users/oweysmomenzada/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [2]:
val_df.head(5)

,id,title,context,question,answers
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"{'text': ['Denver Broncos', 'Denver Broncos', ..."
1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,"{'text': ['Carolina Panthers', 'Carolina Panth..."
2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"{'text': ['Santa Clara, California', 'Levi's S..."
3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ..."
4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta..."


In [3]:
def create_format(df):
    df = df.copy()
    df.drop(columns=["title"], inplace=True)
    
    answers = [i['text'] for i in list(df["answers"])]
    question = list(df["question"])
    context = list(df["context"])
    id_ = list(df["id"])
    
    json_list = []

    for i in range(len(context)):
        res = {
                "context":context[i],
                "qas":[{
                    "question":question[i],
                    "answers":answers[i]
                }]
            }

        json_list.append(res)

    header = [{"header": {"dataset": "SQuAD", "split": "dev"}}]
    
    return header, json_list

In [4]:
header, json_list = create_format(val_df)

In [5]:
with open("dev.jsonl", 'w') as f:
    f.write(json.dumps(header) + "\n")
    
    for item in json_list:
        f.write(json.dumps(item) + "\n")

#### Now we create the trainset

Here, we will use randomseeds of size $1024$ from the SQuAD trainset. Note, that this will be our endpoint, where the user can send new trainable data. 

In [6]:
dataset = load_dataset("squad", split="train")
train_df = pd.DataFrame(dataset).sample(n=1024, random_state=42)
train_df.reset_index(inplace=True)

Found cached dataset squad (/Users/oweysmomenzada/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [7]:
train_df.head(5)

,index,id,title,context,question,answers
0,9983,56de4d9ecffd8e1900b4b7e2,Institute_of_technology,The world's first institution of technology or...,What year was the Banská Akadémia founded?,"{'text': ['1735'], 'answer_start': [167]}"
1,43267,572674a05951b619008f7319,Film_speed,The standard specifies how speed ratings shoul...,What is another speed that can also be reporte...,"{'text': ['SOS-based speed'], 'answer_start': ..."
2,81021,5730bb058ab72b1400f9c72c,Sumer,The most impressive and famous of Sumerian bui...,Where were the use of advanced materials and t...,"{'text': ['Sumerian temples and palaces'], 'an..."
3,49374,572781a5f1498d1400e8fa1f,"Ann_Arbor,_Michigan",Ann Arbor has a council-manager form of govern...,Who is elected every even numbered year?,"{'text': ['mayor'], 'answer_start': [192]}"
4,53414,572843ce4b864d190016485c,John_von_Neumann,"Shortly before his death, when he was already ...",What was the purpose of top secret ICBM commit...,{'text': ['decide on the feasibility of buildi...


In [8]:
header, json_list = create_format(train_df)

In [9]:
with open("squad-train-seed-42-num-examples-1024.jsonl", 'w') as f:
    f.write(json.dumps(header) + "\n")
    
    for item in json_list:
        f.write(json.dumps(item) + "\n")

In [19]:
import torch

# smaller sample with 124 entries
small_train_df = pd.DataFrame(dataset).sample(n=128, random_state=42)
small_train_df.reset_index(inplace=True)

with open("data/data-train-seed-42-num-examples-128.jsonl", 'w') as f:
    f.write(json.dumps(header) + "\n")
    
    for item in json_list:
        f.write(json.dumps(item) + "\n")